# FugueSQL Python



## Jinja Templating

Instead of creating the DataFrame inside the FugueSQLWorkflow, actual applications will often require loading in the DataFrame. This can be mixed with the programming interface.